Introduction
============

Recently, I started using the [pandas](http://pandas.pydata.org/) python library to improve the quality 
(and quantity) of statistics in my applications. One method that has helped me much is the `pivot_table`
command. This is a really (at least for me) complicated command that has
[very poor documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html).
Beyond this, this command is explained a little more in an article about 
[data reshaping](http://pandas.pydata.org/pandas-docs/stable/reshaping.html), however, even this leaves
much to be desired (when I first tried reading it I was overwhelmed by the amount of information there).

A great introduction to pandas is the [three part series](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/) by Greg Reda - it touches `pivot_table` however I was only able to understand what it happend *after* I loose a lot of time playing myself with `pivot_table`.

I have to mention that I am no expert in statistics or
numeric analysis so this post won't have any advanced 
information and may even point out some obvious things. However
keep in mind things that may seem obvious to some
experts are really difficult to grasp for a non-expert.

Before continuing, please notice that this article has been written as a [jupyter notebook](http://jupyter.org/) and was integrated with pelican using the [pelican-ipynb plugin](https://github.com/danielfrg/pelican-ipynb). I had to do some modifications to my theme to better integrate the notebook styling, however some stuff may not look as nice as the other articles.

The DataFrame
=============

The most important (new) data structure that pandas uses is the [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). This can be thought as a two dimensional array, something like an Excel sheet. In the pandas nomenclature, the rows of that two-dimensional array are called *indexes* (while the columns keep their name). Also, if we horizontally/vertically pick the values of a single row(index)/column we'll be left with a different data structure called [Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) - so an array is a series. There's also a [Panel](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.html) data structure which is 3-dimensional, more like a complete Excel workbook (the third dimension being the sheets of the workbook) but I won't cover that here.

More info on the above can be found on the corresponding [article about data structures](http://pandas.pydata.org/pandas-docs/stable/dsintro.html).

There are various ways to read the data for a Series or DataFrame: Initializing through arrays or dicts, reading from csv, xls, database, combinining series to create an array and various others. I won't go into any details about this but will include some examples on how to create Series and DataFrames.

Before continuing with more info on the `pivot_table` operation, I will list a number of examples as a quick reference on how to work with Series and DataFrames that will be useful in the rest of the article (and should be useful in general). Notice that I'm using pandas 0.18.1.

Using Series
------------

In [5]:
import pandas as pd

def t(o):
    # Return the class name of the object
    return o.__class__.__name__

# Use an array to create a Series
series1 = pd.Series([10,20,30])
print "series1 (", t(series1), ')\n', series1
# Notice that the index names were automatically generated as 0,1,2
# Use a dict to create a Series
# notice that the keys of the dict will be the index names
series2 = pd.Series({'row1':11,'row2':22,'row3':33})
print "series2 (", t(series2), ')\n', series2

series1 ( Series )
0    10
1    20
2    30
dtype: int64
series2 ( Series )
row1    11
row2    22
row3    33
dtype: int64


In [7]:
# Get values from series using
series2['row1']
# Can also use slicing and interesting operations 
# like array in array [[]] to select specific indexes
print series1[1:]
print series1[[0,2]]
print series2['row2':]
print series2[['row1', 'row3']]

1    20
2    30
dtype: int64
0    10
2    30
dtype: int64
row2    22
row3    33
dtype: int64
row1    11
row3    33
dtype: int64


Using DataFrames
----------------

In [8]:
# Create a DataFrame using a two-dimensional array
# Notice that the indexes and column names were automatically generated
df1 = pd.DataFrame([[10,20,30], [40,50,60]])
print "Dataframe from array: df1(", t(df1), ')'
print df1

# Use a dict to give names to 
df2 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col1':40,'col2':50,'col3':60}])
print "Dataframe from dict: df2(", t(df2), ')'
print df2

# Give names to indexes
df3 = pd.DataFrame([
    {'col1':10,'col2':20,'col3':30}, 
    {'col1':40,'col2':50,'col3':60}
], index=['idx1', 'idx2'])
print "Dataframe from dict, named indexes: df3(", t(df3), ')'
print df3

# What happens when columns are missing
df4 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col2':40,'col3':50,'col4':60}])
print "Dataframe from dict, missing columns: df4(", t(df4), ')'
print df4

# Create a DataFrame by combining series
df5 = pd.DataFrame([pd.Series([1,2]), pd.Series([3,4])], index=['a', 'b'], )
print "Dataframe from series: df5(", t(df5), ')'
print df5

Dataframe from array: df1( DataFrame )
    0   1   2
0  10  20  30
1  40  50  60
Dataframe from dict: df2( DataFrame )
   col1  col2  col3
0    10    20    30
1    40    50    60
Dataframe from dict, named indexes: df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Dataframe from dict, missing columns: df4( DataFrame )
   col1  col2  col3  col4
0  10.0    20    30   NaN
1   NaN    40    50  60.0
Dataframe from series: df5( DataFrame )
   0  1
a  1  2
b  3  4


In [9]:
print "df3(", t(df3), ")\n", df3

# Pick values from a dataframe using array indexing
print "Get value\n", df3['col2']['idx2']

# We can get a column as a series
print "Get column as series\n", df3['col3']

# Or multiple columns (as a DatFrame)
print "Get column as series\n", df3[['col3', 'col2']]

# We can also get the column by its idx
print "Get column by index\n", df3[df3.columns[2]]

# Pick an index (select horizontal line) as a series
print "Get index as a series\n", df3.loc['idx1']

# Also can pick by index number
print "Get index as a series by index\n", df3.iloc[0]

df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Get value
50
Get column as series
idx1    30
idx2    60
Name: col3, dtype: int64
Get column as series
      col3  col2
idx1    30    20
idx2    60    50
Get column by index
idx1    30
idx2    60
Name: col3, dtype: int64
Get index as a series
col1    10
col2    20
col3    30
Name: idx1, dtype: int64
Get index as a series by index
col1    10
col2    20
col3    30
Name: idx1, dtype: int64


Modifying DataFrames
--------------------

In [28]:
# Let's copy because some of the following operators change the dataframes
df = df3.copy()
print df

print "Change values of a column"
df['col1'] = [11,41]
print df

print "Change values of an index"
df.loc['idx1'] = [11,21, 31]
print df

print "Add another column to an existing dataframe (changes DataFrame)"
df['col4'] = [1,2]
print df

print "Add another row (index) to an existing dataframe (changes DataFrame)"
df.loc['idx3']=[100,200,300,400]
print df

print "Drop a row (returns new object)"
print df.drop('idx1')

print "Drop a column (returns new object)"
print df.drop('col1', axis=1)

print "Rename index (returns new object)"
print df.rename(index={'idx1': 'new-idx-1'})

print "Rename column (returns new object)"
print df.rename(columns={'col1': 'new-col-1'})

print "Transpose array- change columns to rows and vice versa"
print df.T

print "Double transpose - returns the initial DataFrame"
print df.T.T

      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Change values of a column
      col1  col2  col3
idx1    11    20    30
idx2    41    50    60
Change values of an index
      col1  col2  col3
idx1    11    21    31
idx2    41    50    60
Add another column to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
Add another row (index) to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
idx3   100   200   300   400
Drop a row (returns new object)
      col1  col2  col3  col4
idx2    41    50    60     2
idx3   100   200   300   400
Drop a column (returns new object)
      col2  col3  col4
idx1    21    31     1
idx2    50    60     2
idx3   200   300   400
Rename index (returns new object)
           col1  col2  col3  col4
new-idx-1    11    21    31     1
idx2         41    50    60     2
idx3        100   200  

More advanced operations
------------------------

Beyond the previous, more or less basic operations, pandas allows you to do some more advanced operations like SQL-like joins of more than one dataset or, applying a function to each of the rows / columns or even individual cells of the DataFrame:

In [90]:
authors_df=pd.DataFrame([{'id': 1, 'name':'Stephen King'}, {'id': 2, 'name':'Michael Crichton'}],  )

books_df=pd.DataFrame([
        {'id': 1, 'author_id':1, 'name':'It'}, 
        {'id': 2, 'author_id':1, 'name':'The Stand'}, 
        {'id': 3, 'author_id':2, 'name':'Airframe'},
        {'id': 4, 'author_id':2, 'name':'Jurassic Park'}
    ])

print authors_df
print books_df
print books_df.merge(authors_df, left_on='author_id', right_on='id')

   id              name
0   1      Stephen King
1   2  Michael Crichton
   author_id  id           name
0          1   1             It
1          1   2      The Stand
2          2   3       Airframe
3          2   4  Jurassic Park
   author_id  id_x         name_x  id_y            name_y
0          1     1             It     1      Stephen King
1          1     2      The Stand     1      Stephen King
2          2     3       Airframe     2  Michael Crichton
3          2     4  Jurassic Park     2  Michael Crichton


As can be seen above, the `merge` method of DataFrame can be used to do an sql-like join with another DataFrame, using specific columns as join-keys for each of the two dataframes (`left_on` and `right_on`). There are a lot of options for doing various join types (left, right, inner, outer etc) and concatenating DataFrames with other ways - most are discussed in the [corresponding post](http://pandas.pydata.org/pandas-docs/stable/merging.html).

Let's see another method of doing the above join that is more controlled, using the `apply` method of DataFrame that *applies* a function to each row/column of the DataFrame and returns the result as a series:

In [94]:
# Let's do the join using a different method
def f(r):
    author_df_partial = authors_df[authors_df.id==r['author_id']]
    
    return author_df_partial.iloc[0]['name']
    
books_df['author name'] = books_df.apply(f, axis=1)
print books_df

   author_id  id           name       author name
0          1   1             It      Stephen King
1          1   2      The Stand      Stephen King
2          2   3       Airframe  Michael Crichton
3          2   4  Jurassic Park  Michael Crichton


How does this work? We pass the `axis=1` parameter to `apply` so that the callback function will be called for each row of the DataFrame (by default `axis=0` which means it will be called for each column). So, `f` will be called getting each row as an input. From this `book_df` row, we get the `author_id` it contains and filter `authors_df` by it. Notice that `author_df_partial` is actually a DataFrame containing only one row, so we need to filter it by getting its only line, using `iloc[0]` which will return a Series and finally, we return the author name using the corresponding index.

Introducing `pivot_table`
=========================

The `pivot_table` methods is applied to a DataFrame and its purpose is to "reshape" the DataFrame. More on reshaping can be found [here](http://pandas.pydata.org/pandas-docs/stable/reshaping.html). Reshaping means that we want to change the indexes/columns of the DataFrame  while aggregating the values.

Let's start by creating a nice set of data we'll use for the `pivot_table` operations:

In [231]:
books_df=pd.DataFrame([
    {'author':'Stephen King', 'name':'It', 'pages': 1138, 'year': 1986, 'genre': 'Horror',},  
    {'author':'Stephen King', 'name':'The Stand', 'pages': 823, 'year': 1978, 'genre': 'Horror',}, 
    {'author':'Stephen King', 'name': 'Salem\'s Lot', 'pages': 439, 'year': 1975, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Misery', 'pages': 320, 'year': 1987, 'genre': 'Thriller',},
    {'author':'Stephen King', 'name': 'Pet Sematary', 'pages': 374, 'year': 1983, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Bag of bones', 'pages': 529, 'year': 1998, 'genre': 'Horror',},
    {'author':'Stephen King', 'name': 'Different Seasons', 'pages': 527, 'year': 1982, 'genre': 'Thriller',},
    {'author':'Stephen King', 'name': 'The Dark Tower: The Gunslinger', 'pages': 224, 'year': 1982, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower II: The Drawing of the Three', 'pages': 400, 'year': 1987, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower III: The Waste Lands', 'pages': 512, 'year': 1991, 'genre': 'Fantasy',},
    {'author':'Stephen King', 'name': 'The Dark Tower IV: Wizard and Glass', 'pages': 787, 'year': 1998, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Airframe', 'pages': 352, 'year': 1996, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'Jurassic Park', 'pages': 448, 'year':1990, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Congo', 'pages': 348, 'year':1980, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Sphere', 'pages': 385, 'year':1987, 'genre': 'Fantasy',},
    {'author':'Michael Crichton', 'name':'Rising Sun', 'pages': 385, 'year':1992, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'Disclosure ', 'pages': 597, 'year':1994, 'genre': 'Crime',},
    {'author':'Michael Crichton', 'name':'The Lost World ', 'pages': 430, 'year':1995, 'genre': 'Fantasy',},
    {'author':'John Grisham', 'name':'A Time to Kill', 'pages': 515, 'year':1989, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Firm', 'pages': 432, 'year':1991, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Pelican Brief', 'pages': 387, 'year':1992, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Chamber', 'pages': 496, 'year':1994, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Rainmaker', 'pages': 434, 'year':1995, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Runaway Jury', 'pages': 414, 'year':1996, 'genre': 'Crime',},
    {'author':'John Grisham', 'name':'The Street Lawyer', 'pages': 347, 'year':1998, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'Nick\'s Trip ', 'pages': 276, 'year':1993, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'A Firing Offense', 'pages': 216, 'year':1992, 'genre': 'Crime',},
    {'author':'George Pelecanos', 'name':'The Big Blowdown', 'pages': 313, 'year':1996, 'genre': 'Crime',},
    {'author':'George R.R Martin', 'name':'A Clash of Kings', 'pages': 768, 'year':1998, 'genre': 'Fantasy',},
    {'author':'George R.R Martin', 'name':'A Game of Thrones', 'pages': 694, 'year':1996, 'genre': 'Fantasy',},
])
books_df.sort_values(by='year')


# Add a decade column to the books DataFrame
def add_decade(y):
        return str(y['year'])[2] + '0s\''
    
books_df['decade'] = books_df.apply(add_decade, axis=1)

# Add a size column to the books DataFrame
def add_size(y):
        if y['pages'] > 600:
            return 'big'
        elif y['pages'] < 300:
            return 'small'
        return 'medium'
    
books_df['size'] = books_df.apply(add_size, axis=1)
books_df


,author,genre,name,pages,year,decade,size
0,Stephen King,Horror,It,1138,1986,80s',big
1,Stephen King,Horror,The Stand,823,1978,70s',big
2,Stephen King,Horror,Salem's Lot,439,1975,70s',medium
3,Stephen King,Thriller,Misery,320,1987,80s',medium
4,Stephen King,Horror,Pet Sematary,374,1983,80s',medium
5,Stephen King,Horror,Bag of bones,529,1998,90s',medium
6,Stephen King,Thriller,Different Seasons,527,1982,80s',medium
7,Stephen King,Fantasy,The Dark Tower: The Gunslinger,224,1982,80s',small
8,Stephen King,Fantasy,The Dark Tower II: The Drawing of the Three,400,1987,80s',medium
9,Stephen King,Fantasy,The Dark Tower III: The Waste Lands,512,1991,90s',medium


The recommended type of input (at least by me) to the `pivot_table` is a simple DataFrame like the one I have already created:  Your index will be the `id` of your database (or you could even have an auto-generated index like in the example) and the columns will be the values you want to create representations on. This is very easy to create either by reading a file (xls/csv) or by a simple SQL query (substituting all foreign keys with a representative value).

The [pivot_table method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) user four basic parameters:

* `index`: An array of the columns or index names that will be used as indexes to the resulting DataFrame
* `columns`: An array of the columns or index names that will be used as a columns to the resulting DataFrame
* `values`: An array of the values we want to aggregate
* `aggfunc`: Which is the function (or functions) that will be used for aggregating the values

So, how it actually works? You select a number of your columnns (let's suppose that you have followed my recommendation and your data has the type I'd already described) and assign them to either `index` or `columns`, depending if you want to put them horizontally or vertically -- notice that both `index` and `columns` take either a string (to denote a single column) or an array to denote multiple columns -- multiple columns means that you'll have [hierachical indexes / columns](http://pandas.pydata.org/pandas-docs/stable/advanced.html) in your pivot. Hierarchical indexes / columns mean that, the rows/columns would be grouped by a hierarchy, depending on where they actually belong. Let's make it crystal with an example:

If we used `'decade'` as an index, then the pivot_table index would be like 
* `70s` value1 value2 ...
* `80s` value1 value2 ...
* `90s` value1 value2 ...

while, if we used `['decade', 'year']` we'd hove something like

* `70s`
    * `1975` value1 value2 ...
    * `1978` value1 value2 ...
* `80s`
    * `1980` value1 value2 ...
    * `1982` value1 value2 ...
    * ...
* `90s`
    * `1990` value1 value2 ...
    * ... 
    
So, each year would automatically be grouped to its corresponing decade. The same would be true if we used `['decade', 'year']` in columns. Also, notice that pandas doesn't know if the values have any parent / child relationship but just goes from left to right from the most to the less general index. For example, if we had used `['year', 'decade']`, we'd get something like:

* `1975 70s'` value1 value2 ... 
* `1978 70s'` value1 value2 ...
* `1980 80s'` value1 value2 ...	
* `1982 80s'` value1 value2 ...
    
I hope the above clears out how index and columns are used to create the headers for rows and index of `pivot_table`. Please notice that the intersection of index and columns sets have to be an empty set (i.e you can't use the same thing in both).

Now let's talk about the values that the pivot table will contain. 


In [260]:
def make_list(x ):
    print x, type(x)
    return  tuple(x)
        
books_df.pivot_table(index=['year', 'decade'], columns=['size'], values=['author'], aggfunc = make_list)

2    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
1    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
13    Michael Crichton
Name: author, dtype: object <class 'pandas.core.series.Series'>
6    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
7    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
4    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
0    Stephen King
Name: author, dtype: object <class 'pandas.core.series.Series'>
3         Stephen King
8         Stephen King
14    Michael Crichton
Name: author, dtype: object <class 'pandas.core.series.Series'>
18    John Grisham
Name: author, dtype: object <class 'pandas.core.series.Series'>
12    Michael Crichton
Name: author, dtype: object <class 'pandas.core.series.Series'>
9     Stephen King
19    John Grisham
Name: author, dtype: object <class 'pandas.core.series.Series'>
15    Michael Cr

author  \
size                                       big   
year decade                                      
1975 70s'                                 None   
1978 70s'                      (Stephen King,)   
1980 80s'                                 None   
1982 80s'                                 None   
1983 80s'                                 None   
1986 80s'                      (Stephen King,)   
1987 80s'                                 None   
1989 80s'                                 None   
1990 90s'                                 None   
1991 90s'                                 None   
1992 90s'                                 None   
1993 90s'                                 None   
1994 90s'                                 None   
1995 90s'                                 None   
1996 90s'                 (George R.R Martin,)   
1998 90s'    (Stephen King, George R.R Martin)   

                                                                \
size                                                    medium   
year decade                                                      
1975 70s'                                      (Stephen King,)   
1978 70s'                                                 None   
1980 80s'                                  (Michael Crichton,)   
1982 80s'                                      (Stephen King,)   
1983 80s'                                      (Stephen King,)   
1986 80s'                                                 None   
1987 80s'       (Stephen King, Stephen King, Michael Crichton)   
1989 80s'                                      (John Grisham,)   
1990 90s'                                  (Michael Crichton,)   
1991 90s'                         (Stephen King, John Grisham)   
1992 90s'                     (Michael Crichton, John Grisham)   
1993 90s'                                                 None   
1994 90s'                     (Michael Crichton, John Grisham)   
1995 90s'                     (Michael Crichton, John Grisham)   
1996 90s'    (Michael Crichton, John Grisham, George Peleca...   
1998 90s'                         (Stephen King, John Grisham)   

                                  
size                       small  
year decade                       
1975 70s'                   None  
1978 70s'                   None  
1980 80s'                   None  
1982 80s'        (Stephen King,)  
1983 80s'                   None  
1986 80s'                   None  
1987 80s'                   None  
1989 80s'                   None  
1990 90s'                   None  
1991 90s'                   None  
1992 90s'    (George Pelecanos,)  
1993 90s'    (George Pelecanos,)  
1994 90s'                   None  
1995 90s'                   None  
1996 90s'                   None  
1998 90s'                   None

In [261]:
#books_df.pivot_table()

A real-world example
====================

To continue with a real-world example, I will use [MovieLens 100k](http://grouplens.org/datasets/movielens/) to represent some `pivot_table` operations. To load the data I've used the code already provided by the [three part series I already mentioned](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)

In [97]:
import os
import pandas as pd

path = 'C:/Users/serafeim/Downloads/ml-100k' # Change this to your own directory
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(os.path.join(path, 'u.user'), sep='|', names=u_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(os.path.join(path, 'u.data'), sep='\t', names=r_cols, encoding='latin-1')
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv(os.path.join(path, 'u.item'), sep='|', names=m_cols, usecols=range(5), encoding='latin-1')
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [99]:
lens['movie_id']

0           1
1           4
2           5
3           7
4           8
5           9
6          11
7          12
8          15
9          17
10         19
11         21
12         22
13         23
14         24
15         25
16         28
17         30
18         31
19         32
20         42
21         44
22         45
23         47
24         48
25         49
26         50
27         54
28         55
29         56
         ... 
99970     332
99971     334
99972     338
99973     346
99974     682
99975     873
99976     877
99977     886
99978    1527
99979     272
99980     288
99981     294
99982     300
99983     310
99984     313
99985     322
99986     328
99987     333
99988     338
99989     346
99990     354
99991     362
99992     683
99993     689
99994     690
99995     748
99996     751
99997     879
99998     894
99999     901
Name: movie_id, dtype: int64

In [263]:
droplens = lens.drop(['video_release_date', 'imdb_url'], 1)
droplens['release_year']=droplens['release_date'].apply(lambda x: str(x).split('-')[2] if len(str(x).split('-'))>2 else 0)
droplens.head()

,movie_id,title,release_date,user_id,rating,unix_timestamp,age,sex,occupation,zip_code,release_year
0,1,Toy Story (1995),01-Jan-1995,308,4,887736532,60,M,retired,95076,1995
1,4,Get Shorty (1995),01-Jan-1995,308,5,887737890,60,M,retired,95076,1995
2,5,Copycat (1995),01-Jan-1995,308,4,887739608,60,M,retired,95076,1995
3,7,Twelve Monkeys (1995),01-Jan-1995,308,4,887738847,60,M,retired,95076,1995
4,8,Babe (1995),01-Jan-1995,308,5,887736696,60,M,retired,95076,1995


In [264]:
droplens[droplens['release_year'] > '1995'].pivot_table(
    columns=['rating', 'sex',],
    index=[ 'release_year'],
    values=[ 'age'], 
    fill_value=0,
    margins=True 
)

age                                              \
rating                1                     2                     3   
sex                   F          M          F          M          F   
release_year                                                          
1996          30.486239  30.373198  32.781437  31.576959  33.086124   
1997          30.333333  30.795053  31.926647  33.264409  32.638767   
1998          25.105263  31.028169  34.205882  36.059524  32.360000   
All           30.181609  30.593199  32.400000  32.404891  32.857241   

                                                                                
rating                           4                     5                   All  
sex                   M          F          M          F          M             
release_year                                                                    
1996          32.718188  32.757335  32.178464  30.494777  32.073340  32.126852  
1997          33.808467  33.224982  33.826004  32.225541  33.025606  33.086874  
1998          34.153061  32.845238  33.860656  33.785714  34.089431  33.403633  
All           33.208988  32.970278  32.986571  31.375121  32.611830  32.598420

In [265]:
droplens[droplens['release_year'] > '1995'].pivot_table(index=['rating'], values=['age'], columns=['sex'] ).dtypes

     sex
age  F      float64
     M      float64
dtype: object

How to use the pivot!

In [266]:
droplens[droplens['release_year'] > '1995'].head().pivot(index='movie_id', columns='age', values='rating')

age,60
movie_id,
15,3
17,4
21,3
22,4
23,5
